# Approach:


1.   We use AllenNLP and Spacy to extract sentence segments
2.   We classify potential answers for the 5 w questions
3.   We then post process the resulting sentences
4.   We train and use a clf on the sentence embeddings of the verbs to classify where they qualify to be events



In [ ]:
!pip install normalization

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for normalization: filename=normalization-0.4-cp37-none-any.whl size=44015 sha256=d3651fa09a0345146c86e260640218c4be85f69fbac3e0394364a5dab878b20a
  Stored in directory: /root/.cache/pip/wheels/41/82/64/5efe898b7aaf4ba436f813c747e1c7586fc49341e2d689c8de
Successfully built normalization


In [ ]:
import nltk
import pandas as pd


In [ ]:
nltk.download(["brown","punkt", "averaged_perceptron_tagger", "maxent_ne_chunker", "words"])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
# sample from Pippi Longstocking.
text = """
Way out at the end of a tiny little town was an old overgrown garden, and in the garden was an old house, and in the house lived Pippi Longstocking. She was nine years old, and she lived there all alone. She had no mother and no father, and that was of course very nice because there was no one to tell her to go to bed just when she was having the most fun, and no one who could make her take cod liver oil when she much preferred caramel candy.
Once upon a time Pippi had had a father of whom she was extremely fond. Naturally she had had a mother too, but that was so long ago that Pippi didn't remember her at all. Her mother had died when Pippi was just a tiny baby and lay in a cradle and howled so that nobody could go anywhere near her. Pippi was sure that her mother was now up in Heaven, watching her little girl through a peephole in the sky, and Pippi often waved up at her and called, "Don't you worry about me. I'll always come out on top."
Pippi had not forgotten her father. He was a sea captain who sailed on the great ocean, and Pippi had sailed with him in his ship until one day her father was blown overboard in a storm and disappeared. But Pippi was absolutely certain that he would come back. She would never believe that he had drowned; she was sure he had floated until he landed on an island inhabited by cannibals. And she thought he had become the king of all the cannibals and went around with a golden crown on his head all day long.
"My papa is a cannibal king; it certainly isn't every child who has such a stylish papa," Pippi used to say with satisfaction. "And as soon as my papa has built himself a boat he will come and get me, and I'll be a cannibal princess. Heigh- ho, won't that be exciting!"
Her father had bought the old house in the garden many years ago. He thought he would live there with Pippi when he grew old and couldn't sail the seas any longer. And then this annoying thing had to happen, that he was blown into the ocean, and while Pippi was waiting for him to come back she went straight home to Villa Villekulla. That was the name of the house. It stood there ready and waiting for her. One lovely summer evening she had said good-by to all the sailors on her father's boat. They were all fond of Pippi, and she of them.
"So long, boys." she said and kissed each one on the forehead. "Don't you worry about me. I'll always come out on top."
Two things she took with her from the ship: a little monkey whose name was Mr. Nilsson--he was a present from her father--and a big suitcase full of gold pieces. The sailors stood upon the deck and watched as long as they could see her. She walked straight ahead without looking back at all, with Mr. Nilsson on her shoulder and her suitcase in her hand.
"A remarkable child," said one of the sailors as Pippi disappeared in the distance.
He was right. Pippi was indeed a remarkable child. The most remarkable thing about her was that she was so strong. She was so very strong that in the whole wide world there was not a single police officer as strong as she. Why, she could lift a whole horse if she wanted to! And she wanted to. She had a horse of her own that she had bought with one of her many gold pieces the day she came home to Villa Villekulla. She had always longed for a horse, and now here he was, living on the porch. When Pippi wanted to drink her afternoon coffee there, she simply lifted him down into the garden.
Beside Villa Villekulla was another garden and another house. In that house lived a father and mother and two charming children, a boy and a girl. The boy's name was Tommy and the girl's Annika. They were good, well brought up, and obedient children. Tommy would never think of biting his nails, and he always did exactly what his mother told him to do. Annika never fussed when she didn't get her own way, and she always looked pretty in her little well- ironed cotton dresses; she took the greatest care not to get than dirty. Tommy and Annika played nicely with each other in their garden, but they had often wished for a playmate. While Pippi was still sailing on the ocean with her father, they often used to hang over the fence and say to each other, "Isn't it silly that nobody ever moves into that house. Somebody ought to live there somebody with children."
It soon became known throughout the little town that a nine-year-old girl was living all by herself in Villa Villekulla, and all the ladies and gentle-men in the town thought this would never do. All children must have someone to advise them, and all children must go to school to learn the multiplication tables. So the ladies and gentle-men decided that the little girl in Villa Villekulla must immediately be placed in a children's home.
One lovely afternoon Pippi had invited Tommy and Annika over for afternoon coffee and pepparkakor. She had spread the party out on the front steps. It was so sunny and beautiful there, and the air was filled with the fragrance of the flowers in Pippi's garden. Mr. Nilsson climbed around on the porch railing, and every now and then the horse stuck out his head so that he'd be invited to have a cookie.
"Oh, isn't it glorious to be alive?" said Pippi, stretching out her legs as far as she could reach.
Just at that moment two police officers in full uniform came in through the gate.
"Hurray!" said Pippi. "This must be my lucky day too! Policemen are the very best things I know. Next to rhubarb pudding." And with her face beaming she went to meet them.
"Is this the girl who has moved into Villa Villekulla?" asked one of the policemen.
"Quite the contrary," said Pippi. "This is a tiny little auntie who lives on the third floor at the other end of the town."
She said that only because she wanted to have a little fun with the policemen, but they didn't think it was funny at all.
They said she shouldn't be such a smarty. And then they went on to tell her that some nice people in the town were arranging for her to get into a children's home.
"I already have a place in a children's home," said Pippi.
"What?" asked one of the policemen. "Has it been arranged already then? What children's home?"
"This one," said Pippi haughtily. "I am a child and this is my home: therefore it is a children's home, and I have room enough here, plenty of room.
"Dear child," said the policeman, smiling, "you don't understand. You must get into a real children's home and have someone look after you."
"Is one allowed to bring horses to your children's home?" asked Pippi.
"No, of course not," said the policeman. "That's what I thought," said Pippi sadly.
"Well, what about monkeys?" "Of course not. You ought to realize that." """

In [ ]:
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9MB 5.4MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=45210e932d5f972cc29ce22a9094ac97c5cea81ea4a51ed790fffb7f55c98a7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-zgy86zq3/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
!python -m spacy download en_core_web_lg

2021-07-06 06:22:55.580194: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8MB 17kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install allennlp
!pip install --upgrade spacy==2.0.018
!pip install allennlp-models

     |████████████████████████████████| 686kB 5.3MB/s 
     |████████████████████████████████| 17.4MB 152kB/s 
     |████████████████████████████████| 12.1MB 257kB/s 
     |████████████████████████████████| 133kB 43.2MB/s 
     |████████████████████████████████| 112kB 33.0MB/s 
     |████████████████████████████████| 804.1MB 11kB/s 
     |████████████████████████████████| 2.3MB 37.7MB/s 
     |████████████████████████████████| 266kB 35.0MB/s 
     |████████████████████████████████| 1.2MB 23.9MB/s 
     |████████████████████████████████| 133kB 44.5MB/s 
     |████████████████████████████████| 1.8MB 37.1MB/s 
     |████████████████████████████████| 22.3MB 116kB/s 
     |████████████████████████████████| 174kB 45.6MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 7.7MB 32.5MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 901kB 35.8MB/s 
     |████████████████████████████████| 3.3MB 35.0MB/

     |████████████████████████████████| 25.2MB 90kB/s 
     |████████████████████████████████| 614kB 25.4MB/s 
     |████████████████████████████████| 184kB 41.5MB/s 
     |████████████████████████████████| 153kB 40.6MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 1.9MB 31.6MB/s 
     |████████████████████████████████| 450kB 33.8MB/s 
     |████████████████████████████████| 143kB 42.1MB/s 
  Created wheel for regex: filename=regex-2018.1.10-cp37-cp37m-linux_x86_64.whl size=549130 sha256=498d2c821791c66eda2dcd48eb3d48bfa0f64b1be8ee9d48c34c99d0dd6448e9
  Stored in directory: /root/.cache/pip/wheels/74/17/3f/c77bba99efd74ba1a19862c9dd97f4b6d735e2826721dc00ff
  Created wheel for dill: filename=dill-0.2.9-cp37-none-any.whl size=77421 sha256=6edf82cd28d2697f07b4346674f2799d1b3776c2ae393e7fb790d6a06251a9c5
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Created wheel for cytoolz: f

     |████████████████████████████████| 440kB 4.2MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 12.8MB 270kB/s 
     |████████████████████████████████| 9.1MB 32.5MB/s 
     |████████████████████████████████| 133kB 30.1MB/s 
     |████████████████████████████████| 460kB 29.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 624kB 28.5MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=8be7e4093e6842a9dc794ea8c8ab39aa6c2104a7b4af2718973bd192dff428d0
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
  Created wheel for word2number: filename=word2number-1.1-cp37-none-any.whl size=5584 sha256=d5bfc5339af1c34e460e9a7e0463dcfd69fe3a1c7dad8ee6d2a2db530dab5f39
  Stored in directory: /root/.cache/p

In [ ]:
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githack.com/IvanMatoshchuk/NLP_university/main/DATA/01villa.txt

!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githack.com/IvanMatoshchuk/NLP_university/main/DATA/02thing.txt
     
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githack.com/IvanMatoshchuk/NLP_university/main/DATA/03police.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   191  100   191    0     0    276      0 --:--:-- --:--:-- --:--:--   276
100 12667  100 12667    0     0  10743      0  0:00:01  0:00:01 --:--:-- 45078
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   191  100   191    0     0    279      0 --:--:-- --:--:-- --:--:--   279
100 13082  100 13082    0     0  12295      0  0:00:01  0:00:01 --:--:-- 12295
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   191  100   191    0     0    277      0 --:--:-- --:--:-- --:--:--   276
100  8641  100  8641    0     0   7777      0  0:00:01  0:00:01 --:--:--  7777


In [ ]:
import os
os.listdir()

['.config', '03police.txt', '02thing.txt', '01villa.txt', 'sample_data']

In [ ]:


with open('01villa.txt', 'r') as text:
    text1 = text.read()

with open('02thing.txt', 'r') as text:
    text2 = text.read()

with open('03police.txt', 'r') as text:
    text3 = text.read()


whole_text = text1 + " " + text2 + " " + text3

In [ ]:

from allennlp.predictors.predictor import Predictor
from spacy.language import Language
from spacy.tokens import Doc
@Language.factory("srl", default_config={
    "model_path": "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"})
def create_srl_component(nlp: Language, name: str, model_path: str):
    return SRLComponent(nlp, model_path)
class SRLComponent:
    def __init__(self, nlp: Language, model_path: str):
        if not Doc.has_extension("srl"):
            Doc.set_extension("srl", default=None)
        self.predictor = Predictor.from_path(model_path)
    def __call__(self, doc: Doc):
        predictions = self.predictor.predict(sentence=doc.text)
        doc._.srl = predictions
        return doc
        
if __name__ == '__main__':
    import spacy
    nlp = spacy.blank('en')
    nlp.add_pipe("srl")
    doc = nlp("The dog trashed the apartment in under 30 seconds.")
    print(doc._.srl)

Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
downloading: 100%|##########| 405972254/405972254 [00:08<00:00, 45701391.37B/s]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/spacy/util.py:717: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version r

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
{'verbs': [{'verb': 'trashed', 'description': '[ARG0: The dog] [V: trashed] [ARG1: the apartment] [ARGM-TMP: in under 30 seconds] .', 'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'I-ARGM-TMP', 'O']}], 'words': ['The', 'dog', 'trashed', 'the', 'apartment', 'in', 'under', '30', 'seconds', '.']}


In [ ]:
import json
doc=nlp("He is smart and he did the job very good")
print(doc._.srl)

{'verbs': [{'verb': 'is', 'description': '[ARG1: He] [V: is] [ARG2: smart] and he did the job very good', 'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'verb': 'did', 'description': 'He is smart and [ARG0: he] [V: did] [ARG1: the job] [ARGM-MNR: very good]', 'tags': ['O', 'O', 'O', 'O', 'B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}], 'words': ['He', 'is', 'smart', 'and', 'he', 'did', 'the', 'job', 'very', 'good']}


In [ ]:
import spacy
import en_core_web_lg

nlp2 =en_core_web_lg.load()

In [ ]:
import json
import re 
from tqdm import tqdm


sentences = nltk.sent_tokenize(whole_text)


regex = r'\[(.*?)\]'


final = {'sentence':[],
         'Who':[],
         'What':[],
         'Why':[],
         'Where':[],
         'When':[],
         'How':[]}

for sent in tqdm(sentences):

    final['sentence'].append(sent)
    who = []
    what = []
    why = []
    where = []
    when = []
    how = []

    doc = nlp(sent.lower())

    for i in range(len(doc._.srl['verbs'])):
        #if doc._.srl['verbs'][i]['verb'] in ["is","was","has","had","must","might"]:
        
       # if (doc._.srl['verbs'][i]['verb'][-3:] in ["ing"]) or (doc._.srl['verbs'][i]['verb'] in ["was","has","had","is","will","I'll","must"]):
            
        temp_dict = {}
        #print(doc._.srl['verbs'][i]['verb'])
        output = re.findall(regex, doc._.srl['verbs'][i]['description'])

        
        for arg in output:
            sub_arg = arg.split(":")
            temp_dict[sub_arg[0]] = sub_arg[1]
    
        # print(output)
        # print(sortargs(output))
        # print("\n",temp_dict)

        try:
            if temp_dict["ARG0"] not in who:
                who.append(temp_dict["ARG0"])
            #who.append(temp_dict["ARG1"])
            what.append(temp_dict["V"] + " " + temp_dict["ARG1"])
        except KeyError:

            #print(temp_dict)
            # try:
            #     if temp_dict["ARG1"] not in who:
            #         what.append(temp_dict["V"] + " " + temp_dict["ARG1"])
            # except KeyError:
            #     continue


            pass
        
        try:
            if temp_dict["ARGM-LOC"] not in where:
                where.append(temp_dict["ARGM-LOC"])
        except KeyError:
            pass

        try:
            if temp_dict["ARGM-TMP"] not in when:
                when.append(temp_dict["ARGM-TMP"])
        except KeyError:
            pass

        try:
            if temp_dict["ARGM-PRD"] not in how:
                how.append(temp_dict["ARGM-PRD"])
        except KeyError:
            pass

        try:
            if temp_dict["ARGM-MNR"] not in how:

                how.append(temp_dict["ARGM-MNR"])
        except KeyError:
            pass
        try:
            if temp_dict["ARGM-CAU"] not in why:

                why.append(temp_dict["ARGM-CAU"])
        except KeyError:
            pass
        
    final['Who'].append("| ".join(who))
    final['What'].append("| ".join(what))
    final["Why"].append("| ".join(why))
    final["Where"].append("| ".join(where))
    final["How"].append("| ".join(how))
    final["When"].append("| ".join(when))



100%|██████████| 486/486 [03:15<00:00,  2.49it/s]


In [ ]:
final = pd.DataFrame.from_dict(final)
final

,sentence,Who,What,Why,Where,When,How
0,Way out at the end of a tiny little town was a...,pippi longstocking,,,in the house,,
1,"She was nine years old, and she lived there al...",she,,,there,,all alone
2,"She had no mother and no father, and that was ...","she, no one, her","had no mother and no father, tell to go to...",because there was no one to tell her to go to...,,"just when she was having the most fun, when,...",
3,Once upon a time Pippi had had a father of who...,pippi,had a father of whom she was extremely fond,,,once upon a time,
4,"Naturally she had had a mother too, but that w...","she, pippi","had a mother, remember her",,,,
...,...,...,...,...,...,...,...
481,"""Oh, yes,"" said Annika.",annika,"said oh , yes",,,,
482,"""I don't even remember what we used to play be...","i, we",remember what we used to play before she cam...,,,before she came,
483,"""Oh, sure, we played croquet and things like t...","we, annika","played croquet, said oh , sure , we played...",,,,
484,"""But it's lots more fun with Pippi around, I t...",i,,,,,


In [ ]:
events=final[final.What!='']
events

,sentence,Who,What,Why,Where,When,How
2,"She had no mother and no father, and that was ...","she, no one, her","had no mother and no father, tell to go to...",because there was no one to tell her to go to...,,"just when she was having the most fun, when,...",
3,Once upon a time Pippi had had a father of who...,pippi,had a father of whom she was extremely fond,,,once upon a time,
4,"Naturally she had had a mother too, but that w...","she, pippi","had a mother, remember her",,,,
6,Pippi was sure that her mother was now up in H...,"her mother, pippi, you","watching her little girl, called do n't yo...",,,"now, often",through a peephole in the sky
8,Pippi had not forgotten her father.,pippi,forgotten her father,,,,
...,...,...,...,...,...,...,...
479,The horse trotted round and round in the garde...,"the horse, tommy",sang here come the swedes with a clang and a...,,in the garden,,"round and round, with a clang and a bang"
480,When Tommy and Annika had gone to bed that nig...,"tommy and annika, tommy, you","said annika , do n't you think it 's good th...",,,"that night, when tommy and annika had gone t...",
481,"""Oh, yes,"" said Annika.",annika,"said oh , yes",,,,
482,"""I don't even remember what we used to play be...","i, we",remember what we used to play before she cam...,,,before she came,


In [ ]:
def lemma_verbs(x):
  doc=nlp2(x)
  verbs=[]
  for token in doc:
    if token.pos_=='VERB' and token.lemma_ not in ['be','have']:
      verbs.append(token.text)
  if len(verbs)==0:
    return ''
  else:
    return x
events['What']=events.What.apply(lemma_verbs)
events=events[events.What!='']
events
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,sentence,Who,What,Why,Where,When,How
2,"She had no mother and no father, and that was ...","she, no one, her","had no mother and no father, tell to go to...",because there was no one to tell her to go to...,,"just when she was having the most fun, when,...",
4,"Naturally she had had a mother too, but that w...","she, pippi","had a mother, remember her",,,,
6,Pippi was sure that her mother was now up in H...,"her mother, pippi, you","watching her little girl, called do n't yo...",,,"now, often",through a peephole in the sky
8,Pippi had not forgotten her father.,pippi,forgotten her father,,,,
11,She would never believe that he had drowned; s...,"she, he, by cannibals","believe that he had drowned, inhabited an ...",,on an island inhabited by cannibals,until he landed on an island inhabited by can...,
...,...,...,...,...,...,...,...
479,The horse trotted round and round in the garde...,"the horse, tommy",sang here come the swedes with a clang and a...,,in the garden,,"round and round, with a clang and a bang"
480,When Tommy and Annika had gone to bed that nig...,"tommy and annika, tommy, you","said annika , do n't you think it 's good th...",,,"that night, when tommy and annika had gone t...",
481,"""Oh, yes,"" said Annika.",annika,"said oh , yes",,,,
482,"""I don't even remember what we used to play be...","i, we",remember what we used to play before she cam...,,,before she came,


In [ ]:
!git clone https://github.com/dbamman/ACL2019-literary-events.git

Cloning into 'ACL2019-literary-events'...
remote: Enumerating objects: 449, done.
remote: Total 449 (delta 0), reused 0 (delta 0), pack-reused 449
Receiving objects: 100% (449/449), 8.44 MiB | 4.36 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
import os
from tqdm.notebook import tqdm
main_path='./ACL2019-literary-events/data/tsv/train/'
data=[]
labels=[]
for f in os.listdir(main_path):
  file_path=os.path.join(main_path,f)
  try:
    df=pd.read_table(file_path,header=None)
    doc=nlp2(" ".join(df[0].values))
    for token,label in tqdm(zip(doc,df[1].values)):
      if token.pos_=='VERB':
        data.append(token.text)
        labels.append(1 if label !='O' else 0)
  except pd.errors.ParserError:
    continue

In [ ]:
import gensim.downloader
glove_vectors = gensim.downloader.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import numpy as np
np.count_nonzero(labels)

718

In [ ]:
X_train=[]
y_train=[]
for w,l in zip(data,labels):
  try:
    v=glove_vectors[w]
    X_train.append(v)
    y_train.append(l)
  except KeyError:
    continue


In [ ]:
X_train=np.array(X_train)
y_train=np.array(y_train)
X_train.shape,y_train.shape

((11455, 300), (11455,))

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=20)
X_train=pca.fit_transform(X_train)

In [ ]:

from sklearn.neighbors import KNeighborsClassifier 
clf=KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
clf.score(X_train,y_train)

0.9235268441728502

In [ ]:
def classify_what(x):
  doc=nlp2(x)
  verbs=[0]
  for token in doc:
    if token.pos_=='VERB':
      try:
        v=pca.transform([glove_vectors[token.text]])
        verbs.append(clf.predict(v)[0])
      except KeyError:
        pass
  return np.max(verbs)
events['isEvent']=events.What.apply(classify_what)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
events[events.isEvent==1]

,sentence,Who,What,Why,Where,When,How,isEvent
6,Pippi was sure that her mother was now up in H...,"her mother, pippi, you","watching her little girl, called do n't yo...",,,"now, often",through a peephole in the sky,1
44,"Tommy would never think of biting his nails, a...","tommy, he, his mother, him","think of biting his nails, biting his nail...",,,always,,1
50,They had gone to visit their grandmother for a...,"they, anybody","visit their grandmother, had no idea that ...",,,for a week,,1
52,Just as they were standing there considering w...,they,"considering what they should do, do what, ...",,,"those dull days, just then",,1
129,"Meanwhile they had come out into the kitchen, ...","pippi, we","cried now we 're going to make a pancake , n...",,,meanwhile,,1
166,"""I knew, even while I was sleeping, that somet...","i, he",knew that something exciting was going to ha...,,,"even while i was sleeping, today, as he yan...",,1
171,"""We're going to see the new girl next door,"" s...","we, tommy","see the new girl next door, said we 're go...",,,,,1
182,"""So nice you called,"" she said and shook her a...","you, she","said so nice you called, shook her apron",,,,,1
191,Tommy and Annika sat and watched how she went ...,"tommy and annika, she","watched how she went through the dough , how...",,,,how,1
197,"""I don't know what you are going to do,"" said ...","i, you, pippi","know what you are going to do, do what, k...",,,,,1


In [ ]:
print("total number of events:",events[events.isEvent==1].shape[0])

total number of events: 25


### Future Work:


1.   Use POS features alongside the verb embeddings
2.   Use Oversampling/undersampling to fix dataset imbalance

